In [1]:
import pandas as pd
import re 
import geopandas as gpd
import pipeline_jmidkiff as pipeline

pd.set_option('display.max_rows', 100)

In [2]:
initial_assessments = pd.read_csv(
    'data/Cook_County_Assessor_s_Residential_Property_Characteristics.zip', 
    sep='\t', compression='zip')
# https://datacatalog.cookcountyil.gov/Property-Taxation/Cook-County-Assessor-s-Residential-Property-Charac/bcnq-qi2z
pipeline.show(initial_assessments)

/usr/local/Cellar/jupyterlab/3.0.15_1/libexec/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (60) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,



Shape:
(1995108, 82)

Data Types:
PIN                                                  int64
Property Class                                       int64
Tax Year                                             int64
Neighborhood Code                                    int64
Land Square Feet                                     int64
Town Code                                            int64
Type of Residence                                  float64
Apartments                                         float64
Wall Material                                      float64
Roof Material                                      float64
Rooms                                              float64
Bedrooms                                           float64
Basement                                           float64
Basement Finish                                    float64
Central Heating                                    float64
Other Heating                                      float64
Central Air          

,PIN,Property Class,Tax Year,Neighborhood Code,Land Square Feet,Town Code,Type of Residence,Apartments,Wall Material,Roof Material,...,Location Factor,Garage indicator,Residential share of building,Pure Market Sale,Pure Market Filter,Neigborhood Code (mapping),Square root of lot size,Square root of age,Square root of improvement size,Town and Neighborhood
0,1011000040000,212,2018,12,11055,10,2.0,5.0,1.0,2.0,...,2.084499,1.0,NaN,0,0,12,105.142760,11.618950,43.634848,1012
1,1011000050000,205,2018,12,6534,10,2.0,0.0,1.0,1.0,...,2.079861,1.0,NaN,0,0,12,80.833162,10.677078,39.522146,1012
2,1011000060000,212,2018,12,6534,10,2.0,3.0,2.0,1.0,...,2.076376,1.0,NaN,0,0,12,80.833162,10.440307,40.951190,1012
3,1011000090000,212,2018,12,13571,10,2.0,2.0,2.0,2.0,...,2.061552,1.0,NaN,0,0,12,116.494635,8.717798,67.416615,1012
4,1011000170000,205,2018,12,9027,10,2.0,0.0,1.0,1.0,...,2.096602,0.0,NaN,0,0,12,95.010526,11.090537,39.799497,1012


In [3]:
col_list = ['Property Address', 'PIN', 'Longitude', 'Latitude', 
            'Property Class', 'Wall Material', 
            'Roof Material', 'Repair Condition', 'Renovation', 
            'Prior Tax Year Market Value Estimate (Land)', 
            'Prior Tax Year Market Value Estimate (Building)', 'Land Square Feet', 
            'Building Square Feet', 'Age']

for col in col_list: 
    print(pipeline.describe(initial_assessments[col]))
    print("-" * 30)
# Important features: 
# Percentages of those in block group: distinct Property Class, 
# distinct wall material, distinct roof material, Repair Condition, Renovation

# Counts: 
# Total units in block group. 

# Medians & Means: 
# Prior Year Market Value Estimate (Land), & Building, Land Square Feet, 
# Building Square Feet, 

pipeline.group_count(initial_assessments, 'Age').sort_values(ascending=False)
# Note that the Assessment data imputes '10' for any houses missing their age. 

count                  1991916
unique                 1452076
top       655 W IRVING PARK RD
freq                      1679
Name: Property Address, dtype: object
------------------------------
count    1.995108e+06
mean     1.433242e+13
std      7.911198e+12
min      1.011000e+12
25%      8.144010e+12
50%      1.405211e+13
75%      1.915217e+13
max      3.332302e+13
Name: PIN, dtype: float64
------------------------------
count    1.991085e+06
mean    -8.779745e+01
std      1.581003e-01
min     -8.826351e+01
25%     -8.788149e+01
50%     -8.776527e+01
75%     -8.767502e+01
max     -8.752481e+01
Name: Longitude, dtype: float64
------------------------------
count    1.991085e+06
mean     4.190541e+01
std      1.649300e-01
min      4.146983e+01
25%      4.179062e+01
50%      4.193405e+01
75%      4.203778e+01
max      4.215398e+01
Name: Latitude, dtype: float64
------------------------------
count    1.995108e+06
mean     2.428938e+02
std      4.303183e+01
min      2.000000e+02
25%      

Age
10     60807
62     43847
61     39770
63     39611
64     37510
       ...  
174        1
200        1
203        1
205        1
208        1
Name: Count, Length: 188, dtype: int64

In [4]:
assessments = initial_assessments[col_list]
assessments = gpd.GeoDataFrame(
    assessments, 
    geometry=gpd.points_from_xy(
        assessments['Longitude'], 
        assessments['Latitude']), 
    crs='EPSG:4326')
pipeline.show(assessments)


Shape:
(1995108, 15)

Data Types:
Property Address                                     object
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age                                                   int64
geometry                                           geometry
dtype: object


,Property Address,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age,geometry
0,213 W MAIN ST BARRINGTON,1011000040000,-88.139621,42.153953,212,1.0,2.0,2.0,NaN,52510.0,374930.0,11055,1904.0,135,POINT (-88.13962 42.15395)
1,209 W MAIN ST BARRINGTON,1011000050000,-88.139379,42.153953,205,1.0,1.0,2.0,NaN,31030.0,173970.0,6534,1562.0,114,POINT (-88.13938 42.15395)
2,205 W MAIN ST BARRINGTON,1011000060000,-88.139196,42.153953,212,2.0,1.0,2.0,NaN,31030.0,353260.0,6534,1677.0,109,POINT (-88.13920 42.15395)
3,149 W MAIN ST BARRINGTON,1011000090000,-88.138427,42.153945,212,2.0,2.0,2.0,NaN,64460.0,668790.0,13571,4545.0,76,POINT (-88.13843 42.15395)
4,115 DUNDEE AVE BARRINGTON,1011000170000,-88.140521,42.153671,205,1.0,1.0,3.0,NaN,42870.0,291310.0,9027,1584.0,123,POINT (-88.14052 42.15367)


In [5]:
acs_g = gpd.read_file('data/geo_census.shp')
pipeline.show(acs_g)


Shape:
(2194, 19)

Data Types:
FIPS           float64
Geographic      object
Census Tra       int64
Block Grou       int64
tot_pop          int64
tot_white        int64
tot_black        int64
hh_size        float64
med_income     float64
occ_units        int64
med_built_     float64
med_rent       float64
oo_hsng_un       int64
GEOID           object
perc_white     float64
perc_non_w     float64
perc_black     float64
perc_owner     float64
geometry      geometry
dtype: object


,FIPS,Geographic,Census Tra,Block Grou,tot_pop,tot_white,tot_black,hh_size,med_income,occ_units,med_built_,med_rent,oo_hsng_un,GEOID,perc_white,perc_non_w,perc_black,perc_owner,geometry
0,1.700000e+11,15000US170310101001,10100,1,461,265,108,1.95,NaN,236,NaN,873.0,117,170310101001,0.574837,0.425163,0.234273,0.495763,"POLYGON ((-87.67009 42.02115, -87.67047 42.021..."
1,1.700000e+11,15000US170310101002,10100,2,2424,746,1543,2.26,21827.0,1054,NaN,799.0,53,170310101002,0.307756,0.692244,0.636551,0.050285,"POLYGON ((-87.66950 42.01936, -87.66963 42.019..."
2,1.700000e+11,15000US170310101003,10100,3,1714,1137,427,1.50,54297.0,1073,NaN,1071.0,327,170310101003,0.663361,0.336639,0.249125,0.304753,"POLYGON ((-87.66681 42.01924, -87.66780 42.019..."
3,1.700000e+11,15000US170310102011,10201,1,1706,479,745,2.30,42778.0,712,1962.0,1097.0,224,170310102011,0.280774,0.719226,0.436694,0.314607,"POLYGON ((-87.68234 42.01250, -87.68268 42.012..."
4,1.700000e+11,15000US170310102012,10201,2,3925,2131,1180,2.69,39535.0,1424,1945.0,1152.0,353,170310102012,0.542930,0.457070,0.300637,0.247893,"POLYGON ((-87.67972 42.01392, -87.68003 42.013..."


In [6]:
assessments_in_chi = gpd.sjoin(
    left_df=acs_g.loc[:,['GEOID', 'geometry']], 
    right_df=assessments, how='inner', op='intersects')
pipeline.show(assessments_in_chi)


Shape:
(728543, 17)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
Property Address                                     object
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age   

,GEOID,geometry,index_right,Property Address,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799443,7760 N SHERIDAN RD,11291010331134,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,23470.0,305620.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799444,7550 N SHERIDAN RD,11291010331136,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799436,7746 N SHERIDAN RD,11291010331049,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799445,7750 N SHERIDAN RD,11291010331023,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,29630.0,385880.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799420,7750 N SHERIDAN RD,11291010331034,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15


In [7]:
cols_to_dummify = [
    'Property Class', 'Wall Material', 'Roof Material', 'Repair Condition', 
    'Renovation']
assessments_wide = (pd.get_dummies(
        data=assessments_in_chi, columns=cols_to_dummify, dummy_na=True)
    .drop(columns=['Property Address']))
pipeline.show(assessments_wide)


Shape:
(728543, 49)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age                                                   int64
Property Class_200.0                                  uint8
Property Class_201.0                                  uint8
Property Class_202.0                                  uint8
Property Class_203.0                                  uint8
Property Class_204.0                                  uint8
Proper

,GEOID,geometry,index_right,PIN,Longitude,Latitude,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799443,11291010331134,-87.666674,42.022737,23470.0,305620.0,61111,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799444,11291010331136,-87.666674,42.022737,1180.0,15410.0,61111,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799436,11291010331049,-87.666674,42.022737,1180.0,15410.0,61111,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799445,11291010331023,-87.666674,42.022737,29630.0,385880.0,61111,NaN,...,0,0,1,0,0,0,1,0,0,1
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799420,11291010331034,-87.666674,42.022737,1180.0,15410.0,61111,NaN,...,0,0,1,0,0,0,1,0,0,1


In [8]:
check = assessments_wide == 0
# Drop any columns that are all 0. 
to_drop = list(check.all()[check.all() == True].index)
assessments_wide.drop(columns=to_drop, inplace=True)
print(f'Dropped the following columns that were all 0:\n{to_drop}')

Dropped the following columns that were all 0:
['Property Class_nan']


In [9]:
# So helpful: https://stackoverflow.com/a/47103408/8527838

cols = list(assessments_wide.columns.values)
cols.pop(cols.index('GEOID'))
assessments_wide = assessments_wide[cols + ['GEOID']]
# First series of aggregations (all that don't use sum())
assessments_1a = assessments_wide.groupby(['GEOID']).agg({
    'PIN': 'count', 
    'Prior Tax Year Market Value Estimate (Land)': ['mean', 'median'], 
    'Prior Tax Year Market Value Estimate (Building)': ['mean', 'median'], 
    'Land Square Feet': ['mean', 'median'],
    'Building Square Feet': ['mean', 'median'], 
    'Age': ['mean', 'median']    
})
assessments_1a

PIN Prior Tax Year Market Value Estimate (Land)           \
             count                                        mean   median   
GEOID                                                                     
170310101001   247                                37161.255061  14120.0   
170310101002   148                                28536.081081  17080.0   
170310101003   477                                16814.549266  10540.0   
170310102011   283                                28919.399293  17030.0   
170310102012   739                                25685.236806  17070.0   
...            ...                                         ...      ...   
170318439001    95                                21181.684211  11470.0   
170318439002    71                                31305.211268  27940.0   
170318439003   413                                 5240.314770   2320.0   
170318439004   152                                26058.552632  15000.0   
170318439005   198                                16334.292929   3610.0   

             Prior Tax Year Market Value Estimate (Building)            \
                                                        mean    median   
GEOID                                                                    
170310101001                                   165053.643725  171860.0   
170310101002                                   151666.621622  101180.0   
170310101003                                   130144.486373  121130.0   
170310102011                                   164135.689046  129920.0   
170310102012                                   122439.945873   99730.0   
...                                                      ...       ...   
170318439001                                   152839.789474  150800.0   
170318439002                                   152449.718310  162210.0   
170318439003                                    40202.227603   37270.0   
170318439004                                    76780.723684   81670.0   
170318439005                                    79534.595960   48740.0   

             Land Square Feet          Building Square Feet          \
                         mean   median                 mean  median   
GEOID                                                                 
170310101001     32474.959514   9860.0          3244.918367  2110.0   
170310101002      9868.878378   6246.0          4527.851852  4797.0   
170310101003     18124.790356  14913.0          3361.833333  1494.0   
170310102011      7054.893993   6000.0          2308.084906  1641.0   
170310102012     13236.216509   9334.0          2936.382166  2542.0   
...                       ...      ...                  ...     ...   
170318439001      6474.157895   5069.0          4460.357143  4315.5   
170318439002      4264.859155   4611.0          4538.304348  4085.5   
170318439003     40131.932203  48225.0          6684.000000  6684.0   
170318439004     19798.881579  18810.0          6038.227273  6243.0   
170318439005     11443.520202  10805.0          6320.550000  6532.5   

                     Age         
                    mean median  
GEOID                            
170310101001   51.226721   64.0  
170310101002   93.689189   97.0  
170310101003   78.140461   98.0  
170310102011   70.293286   64.0  
170310102012   72.483085   65.0  
...                  ...    ...  
170318439001   95.757895  108.0  
170318439002  107.436620  113.0  
170318439003   54.159806   48.0  
170318439004   87.065789   89.0  
170318439005   72.434343   87.0  

[2156 rows x 11 columns]

In [10]:
# Second series of aggregations (all that use sum())
assessments_1b = (assessments_wide
    .loc[:,'Property Class_200.0':'GEOID']
    .groupby('GEOID').sum())
assessments_1b

,Property Class_200.0,Property Class_201.0,Property Class_202.0,Property Class_203.0,Property Class_204.0,Property Class_205.0,Property Class_206.0,Property Class_207.0,Property Class_208.0,Property Class_209.0,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310101001,0.0,0.0,0.0,5.0,3.0,13.0,8.0,0.0,0.0,0.0,...,2.0,0.0,198.0,1.0,47.0,1.0,198.0,0.0,0.0,247.0
170310101002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,121.0,0.0,26.0,1.0,121.0,0.0,0.0,148.0
170310101003,0.0,0.0,0.0,1.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,441.0,3.0,33.0,0.0,441.0,1.0,0.0,476.0
170310102011,0.0,0.0,1.0,15.0,4.0,11.0,3.0,0.0,0.0,0.0,...,0.0,0.0,177.0,0.0,105.0,1.0,177.0,0.0,0.0,283.0
170310102012,0.0,1.0,3.0,16.0,4.0,2.0,2.0,0.0,0.0,0.0,...,3.0,2.0,582.0,0.0,156.0,1.0,582.0,0.0,0.0,739.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
170318439001,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.0,0.0,67.0,0.0,26.0,2.0,67.0,0.0,0.0,95.0
170318439002,0.0,0.0,0.0,0.0,0.0,1.0,3.0,1.0,0.0,1.0,...,0.0,0.0,25.0,0.0,46.0,0.0,25.0,1.0,0.0,70.0
170318439003,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,411.0,0.0,2.0,0.0,411.0,0.0,0.0,413.0


In [11]:
# Combine the two different data pieces
assessments_block_group = assessments_1a.join(other=assessments_1b)
pipeline.show(assessments_block_group)


Shape:
(2156, 48)

Data Types:
(PIN, count)                                                   int64
(Prior Tax Year Market Value Estimate (Land), mean)          float64
(Prior Tax Year Market Value Estimate (Land), median)        float64
(Prior Tax Year Market Value Estimate (Building), mean)      float64
(Prior Tax Year Market Value Estimate (Building), median)    float64
(Land Square Feet, mean)                                     float64
(Land Square Feet, median)                                   float64
(Building Square Feet, mean)                                 float64
(Building Square Feet, median)                               float64
(Age, mean)                                                  float64
(Age, median)                                                float64
Property Class_200.0                                         float64
Property Class_201.0                                         float64
Property Class_202.0                                         float64
Pr

/usr/local/Cellar/jupyterlab/3.0.15_1/libexec/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)


,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)","(Building Square Feet, median)","(Age, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310101001,247,37161.255061,14120.0,165053.643725,171860.0,32474.959514,9860.0,3244.918367,2110.0,51.226721,...,2.0,0.0,198.0,1.0,47.0,1.0,198.0,0.0,0.0,247.0
170310101002,148,28536.081081,17080.0,151666.621622,101180.0,9868.878378,6246.0,4527.851852,4797.0,93.689189,...,1.0,0.0,121.0,0.0,26.0,1.0,121.0,0.0,0.0,148.0
170310101003,477,16814.549266,10540.0,130144.486373,121130.0,18124.790356,14913.0,3361.833333,1494.0,78.140461,...,0.0,0.0,441.0,3.0,33.0,0.0,441.0,1.0,0.0,476.0
170310102011,283,28919.399293,17030.0,164135.689046,129920.0,7054.893993,6000.0,2308.084906,1641.0,70.293286,...,0.0,0.0,177.0,0.0,105.0,1.0,177.0,0.0,0.0,283.0
170310102012,739,25685.236806,17070.0,122439.945873,99730.0,13236.216509,9334.0,2936.382166,2542.0,72.483085,...,3.0,2.0,582.0,0.0,156.0,1.0,582.0,0.0,0.0,739.0


In [12]:
# Get the proportions for the columns that should have it, dividing by total residences
assessments_block_group.loc[:,'Property Class_200.0':'Renovation_nan'] = (
    assessments_block_group.loc[:,'Property Class_200.0':'Renovation_nan']
        .divide(other=assessments_block_group[('PIN', 'count')], 
                axis=0))
assessments_block_group.head()

,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)","(Building Square Feet, median)","(Age, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
GEOID,,,,,,,,,,,,,,,,,,,,,
170310101001,247,37161.255061,14120.0,165053.643725,171860.0,32474.959514,9860.0,3244.918367,2110.0,51.226721,...,0.008097,0.000000,0.801619,0.004049,0.190283,0.004049,0.801619,0.000000,0.0,1.000000
170310101002,148,28536.081081,17080.0,151666.621622,101180.0,9868.878378,6246.0,4527.851852,4797.0,93.689189,...,0.006757,0.000000,0.817568,0.000000,0.175676,0.006757,0.817568,0.000000,0.0,1.000000
170310101003,477,16814.549266,10540.0,130144.486373,121130.0,18124.790356,14913.0,3361.833333,1494.0,78.140461,...,0.000000,0.000000,0.924528,0.006289,0.069182,0.000000,0.924528,0.002096,0.0,0.997904
170310102011,283,28919.399293,17030.0,164135.689046,129920.0,7054.893993,6000.0,2308.084906,1641.0,70.293286,...,0.000000,0.000000,0.625442,0.000000,0.371025,0.003534,0.625442,0.000000,0.0,1.000000
170310102012,739,25685.236806,17070.0,122439.945873,99730.0,13236.216509,9334.0,2936.382166,2542.0,72.483085,...,0.004060,0.002706,0.787551,0.000000,0.211096,0.001353,0.787551,0.000000,0.0,1.000000


In [13]:
# Notice I don't export from ACS household median age
# or the 'total' variables. 
final_assessments = (acs_g.loc[:,[
    'GEOID', 'hh_size', 'med_income', 'occ_units', 'med_rent', 
    'oo_hsng_un', 'perc_white', 'perc_non_w', 'perc_black', 'perc_owner', 'tot_pop']]
    .merge(
    right=assessments_block_group, 
    left_on='GEOID', right_index=True))
pipeline.show(final_assessments)


Shape:
(2156, 59)

Data Types:
GEOID                                                         object
hh_size                                                      float64
med_income                                                   float64
occ_units                                                      int64
med_rent                                                     float64
oo_hsng_un                                                     int64
perc_white                                                   float64
perc_non_w                                                   float64
perc_black                                                   float64
perc_owner                                                   float64
tot_pop                                                        int64
(PIN, count)                                                   int64
(Prior Tax Year Market Value Estimate (Land), mean)          float64
(Prior Tax Year Market Value Estimate (Land), median)        float64
(P

,GEOID,hh_size,med_income,occ_units,med_rent,oo_hsng_un,perc_white,perc_non_w,perc_black,perc_owner,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
0,170310101001,1.95,NaN,236,873.0,117,0.574837,0.425163,0.234273,0.495763,...,0.008097,0.000000,0.801619,0.004049,0.190283,0.004049,0.801619,0.000000,0.0,1.000000
1,170310101002,2.26,21827.0,1054,799.0,53,0.307756,0.692244,0.636551,0.050285,...,0.006757,0.000000,0.817568,0.000000,0.175676,0.006757,0.817568,0.000000,0.0,1.000000
2,170310101003,1.50,54297.0,1073,1071.0,327,0.663361,0.336639,0.249125,0.304753,...,0.000000,0.000000,0.924528,0.006289,0.069182,0.000000,0.924528,0.002096,0.0,0.997904
3,170310102011,2.30,42778.0,712,1097.0,224,0.280774,0.719226,0.436694,0.314607,...,0.000000,0.000000,0.625442,0.000000,0.371025,0.003534,0.625442,0.000000,0.0,1.000000
4,170310102012,2.69,39535.0,1424,1152.0,353,0.542930,0.457070,0.300637,0.247893,...,0.004060,0.002706,0.787551,0.000000,0.211096,0.001353,0.787551,0.000000,0.0,1.000000


In [14]:
# Export assessment data block group
final_assessments.to_csv('data/final_assessments_block_group.csv', sep='\t')

In [15]:
# Export assessment data at literal block level using address obfuscation 
# like the water set does. 

pipeline.show(assessments_in_chi)


Shape:
(728543, 17)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
Property Address                                     object
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age   

,GEOID,geometry,index_right,Property Address,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799443,7760 N SHERIDAN RD,11291010331134,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,23470.0,305620.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799444,7550 N SHERIDAN RD,11291010331136,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799436,7746 N SHERIDAN RD,11291010331049,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799445,7750 N SHERIDAN RD,11291010331023,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,29630.0,385880.0,61111,NaN,15
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799420,7750 N SHERIDAN RD,11291010331034,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15


In [16]:
assessments_in_chi['Address Obfuscated'] = (
    assessments_in_chi[['Property Address']]
    .replace(to_replace={
        '\d{2}(?=\s)': 'XX', 
        '(?<!\d)\d{1}(?=\s)': 'X'}, regex=True))
addresses = pipeline.group_count(
    df=assessments_in_chi, groupby='Address Obfuscated').sort_index()
pipeline.show(assessments_in_chi)


Shape:
(728543, 18)

Data Types:
GEOID                                                object
geometry                                           geometry
index_right                                           int64
Property Address                                     object
PIN                                                   int64
Longitude                                           float64
Latitude                                            float64
Property Class                                        int64
Wall Material                                       float64
Roof Material                                       float64
Repair Condition                                    float64
Renovation                                          float64
Prior Tax Year Market Value Estimate (Land)         float64
Prior Tax Year Market Value Estimate (Building)     float64
Land Square Feet                                      int64
Building Square Feet                                float64
Age   

,GEOID,geometry,index_right,Property Address,PIN,Longitude,Latitude,Property Class,Wall Material,Roof Material,Repair Condition,Renovation,Prior Tax Year Market Value Estimate (Land),Prior Tax Year Market Value Estimate (Building),Land Square Feet,Building Square Feet,Age,Address Obfuscated
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799443,7760 N SHERIDAN RD,11291010331134,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,23470.0,305620.0,61111,NaN,15,77XX N SHERIDAN RD
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799444,7550 N SHERIDAN RD,11291010331136,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15,75XX N SHERIDAN RD
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799436,7746 N SHERIDAN RD,11291010331049,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15,77XX N SHERIDAN RD
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799445,7750 N SHERIDAN RD,11291010331023,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,29630.0,385880.0,61111,NaN,15,77XX N SHERIDAN RD
0,170310101001,"POLYGON ((-87.67009 42.02115, -87.67047 42.021...",799420,7750 N SHERIDAN RD,11291010331034,-87.666674,42.022737,299,NaN,NaN,NaN,NaN,1180.0,15410.0,61111,NaN,15,77XX N SHERIDAN RD


In [17]:
# I thought about writing a function since I'm copying this code again, 
# but that would have taken away some of the pipeline.show() functionality
# which is important to see the first time through this notebook.  

# Again, dummify
assessments_block_lit_wide = (pd.get_dummies(
        data=assessments_in_chi, columns=cols_to_dummify, dummy_na=True)
    .drop(columns=['Property Address']))

cols = list(assessments_block_lit_wide.columns.values)
cols.pop(cols.index('Address Obfuscated'))
assessments_block_lit_wide = assessments_block_lit_wide[cols + ['Address Obfuscated']]

# First series of aggregations (all that don't use sum())
assessments_block_lit_1a = assessments_block_lit_wide.groupby(['Address Obfuscated']).agg({
    'PIN': 'count', 
    'Prior Tax Year Market Value Estimate (Land)': ['mean', 'median'], 
    'Prior Tax Year Market Value Estimate (Building)': ['mean', 'median'], 
    'Land Square Feet': ['mean', 'median'],
    'Building Square Feet': ['mean', 'median'], 
    'Age': ['mean', 'median']    
})
assessments_block_lit_1a

# Second series of aggregations (all that use sum())
assessments_block_lit_1b = (assessments_block_lit_wide
    .loc[:,'Property Class_200.0':'Address Obfuscated']
    .groupby('Address Obfuscated').sum())
assessments_block_lit_1b

,Property Class_200.0,Property Class_201.0,Property Class_202.0,Property Class_203.0,Property Class_204.0,Property Class_205.0,Property Class_206.0,Property Class_207.0,Property Class_208.0,Property Class_209.0,...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
Address Obfuscated,,,,,,,,,,,,,,,,,,,,,
100XX S ABERDEEN ST,0.0,0.0,7.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,2.0,0.0,0.0,37.0,0.0,0.0,0.0,0.0,37.0
100XX S ARTESIAN AVE,0.0,0.0,0.0,7.0,1.0,18.0,3.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,30.0,0.0,0.0,0.0,0.0,30.0
100XX S AVENUE L,0.0,2.0,7.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,1.0,31.0,0.0,3.0,0.0,0.0,35.0
100XX S AVENUE M,0.0,3.0,11.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,34.0,0.0,5.0,0.0,0.0,39.0
100XX S AVENUE N,0.0,1.0,7.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,15.0,1.0,2.0,0.0,0.0,18.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
XX W MAPLE ST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
XX W MARQUETTE RD,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,3.0
XX W OAK ST,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,45.0,0.0,1.0,0.0,45.0,0.0,0.0,46.0


In [18]:
# Combine the two different data pieces
assessments_block_lit = assessments_block_lit_1a.join(
    other=assessments_block_lit_1b)

# Get the proportions for the columns that should have it, dividing by total residences
assessments_block_lit.loc[:,'Property Class_200.0':'Renovation_nan'] = (
    assessments_block_lit.loc[:,'Property Class_200.0':'Renovation_nan']
        .divide(other=assessments_block_lit[('PIN', 'count')], 
                axis=0))
final_assessments_block_lit = assessments_block_lit

# # Notice I don't export from ACS household median age
# # or the 'total' variables. 
# final_assessments_block_lit = (acs_g.loc[:,[
#     'GEOID', 'hh_size', 'med_income', 'occ_units', 'med_rent', 
#     'oo_hsng_un', 'perc_white', 'perc_non_w', 'perc_black', 'perc_owner']]
#     .merge(
#     right=assessments_block_lit,
#     left_on='GEOID', right_index=True))
pipeline.show(final_assessments_block_lit)

/usr/local/Cellar/jupyterlab/3.0.15_1/libexec/lib/python3.9/site-packages/pandas/core/reshape/merge.py:648: UserWarning: merging between different levels can give an unintended result (2 levels on the left,1 on the right)
  warnings.warn(msg, UserWarning)



Shape:
(25034, 49)

Data Types:
(PIN, count)                                                   int64
(Prior Tax Year Market Value Estimate (Land), mean)          float64
(Prior Tax Year Market Value Estimate (Land), median)        float64
(Prior Tax Year Market Value Estimate (Building), mean)      float64
(Prior Tax Year Market Value Estimate (Building), median)    float64
(Land Square Feet, mean)                                     float64
(Land Square Feet, median)                                   float64
(Building Square Feet, mean)                                 float64
(Building Square Feet, median)                               float64
(Age, mean)                                                  float64
(Age, median)                                                float64
Property Class_200.0                                         float64
Property Class_201.0                                         float64
Property Class_202.0                                         float64
P

,"(PIN, count)","(Prior Tax Year Market Value Estimate (Land), mean)","(Prior Tax Year Market Value Estimate (Land), median)","(Prior Tax Year Market Value Estimate (Building), mean)","(Prior Tax Year Market Value Estimate (Building), median)","(Land Square Feet, mean)","(Land Square Feet, median)","(Building Square Feet, mean)","(Building Square Feet, median)","(Age, mean)",...,Roof Material_5.0,Roof Material_6.0,Roof Material_nan,Repair Condition_1.0,Repair Condition_2.0,Repair Condition_3.0,Repair Condition_nan,Renovation_1.0,Renovation_2.0,Renovation_nan
Address Obfuscated,,,,,,,,,,,,,,,,,,,,,
100XX S ABERDEEN ST,37,27376.216216,24500.0,80191.081081,80770.0,3650.432432,3267.0,1044.513514,1056.0,76.783784,...,0.054054,0.054054,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,1.0
100XX S ARTESIAN AVE,30,48554.666667,47500.0,213167.000000,204625.0,5111.066667,5000.0,1509.200000,1345.0,68.766667,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.0,0.0,1.0
100XX S AVENUE L,35,24846.285714,25000.0,65371.714286,73960.0,3105.857143,3125.0,2019.812500,2001.5,112.200000,...,0.000000,0.000000,0.085714,0.028571,0.885714,0.000000,0.085714,0.0,0.0,1.0
100XX S AVENUE M,39,26409.743590,25000.0,59992.051282,58410.0,3301.256410,3125.0,1488.411765,1195.0,110.307692,...,0.000000,0.000000,0.128205,0.000000,0.871795,0.000000,0.128205,0.0,0.0,1.0
100XX S AVENUE N,18,26388.888889,25000.0,57832.222222,54750.0,3298.611111,3125.0,1729.687500,1027.0,118.222222,...,0.000000,0.000000,0.111111,0.000000,0.833333,0.055556,0.111111,0.0,0.0,1.0


In [19]:
final_assessments_block_lit[[('PIN', 'count')]].describe()

,"(PIN, count)"
count,25034.000000
mean,29.102141
std,65.096757
min,1.000000
25%,9.000000
50%,22.000000
75%,33.000000
max,3035.000000


In [20]:
# Export assessment data literal block
final_assessments_block_lit.to_csv('data/final_assessments_block_lit.csv', sep='\t')